In [1]:
# 必要的包
import os
import json
# aganet
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
# from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic


# from dotenv import load_dotenv
# load_dotenv(dotenv_path="../.env")

In [2]:
# llm导入
import os
from langchain_community.chat_models import ChatZhipuAI
os.environ["ZHIPUAI_API_KEY"] = "36a142a628aca0e1fe1fa29a5e8fefff.kvXnRyd4sFT24xFZ"
llm = ChatZhipuAI(
    model="glm-4",
    temperature=0,
)

In [3]:
# 那些地方、怎么标注
schema = {
    "properties": {
        "sentiment": {
            "type": "string",
            "enum":["happy", "neutral", "sad"]
        },
        "aggressiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how aggressive the statement is, the higher the number the more aggressive",
        },
        "language": {
            "type": "string",
            "enum": ["spanish", "english", "french", "german", "italian"],
        },
    },
    "required": ["sentiment", "language", "aggressiveness"],
}
chain = create_tagging_chain(schema, llm)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_18876\2213029118.py:20: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` that is available on ChatModels capable of tool calling. See API reference for this function for replacement: <https://api.python.langchain.com/en/latest/chains/langchain.chains.openai_functions.tagging.create_tagging_chain.html> You can read more about `with_structured_output` here: <https://python.langchain.com/v0.2/docs/how_to/structured_output/>. If you notice other issues, please provide feedback here: <https://github.com/langchain-ai/langchain/discussions/18154>
  chain = create_tagging_chain(schema, llm)


In [4]:
temp="amazing!this game is so funny!"
chain.run(temp)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_26712\907709850.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(temp)


OutputParserException: Could not parse function call: 'function_call'

In [5]:
# 如果耀调用网络的话
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.utilities import TextRequestsWrapper
from langchain import LLMMathChain
from langchain.agents import Tool
search = DuckDuckGoSearchAPIWrapper()
requests = TextRequestsWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    ),
    Tool(
        name = "Requests",
        func=requests.get,
        description="Useful for when you to make a request to a URL"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, return_intermediate_steps=True, handle_parsing_errors=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18876\3817642133.py:28: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, return_intermediate_steps=True, handle_parsing_errors=True)


In [6]:
response = agent({"input":"What is the capital of canada?"})
response['output']
# response = agent({"input":"Tell me what the comments are about on this webpage https://news.ycombinator.com/item?id=34425779"})
# response['output']
# response = agent({"input":"Tell me what kind of thing this page can do: https://wetools.cc/random"})
# response['output']



C:\Users\Administrator\AppData\Local\Temp\ipykernel_18876\1105324917.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent({"input":"What is the capital of canada?"})




> Entering new AgentExecutor chain...
This is a simple geography question that I can answer without using any tools. 

Final Answer: Ottawa is the capital of Canada.

> Finished chain.


'Ottawa is the capital of Canada.'